# **Data Engineering Process Notebook**

## Author: Sherry Wang
## Date: 2nd of Nov (edited on 4th & 6th of Nov)
---

The purpose of this section is to load in the datasets required in addressing the driving question and conduct data engineering to prepare the datasets ready for the analysis.

In total, there are 6 different datasets that will be imported for the data analysis. The main dataset used in this notebook is sourced from [Our World in Data](https://ourworldindata.org/coronavirus). And supplementary data is from [John Hopkins University](https://coronavirus.jhu.edu/map.html) specifically to obtain the numbers of recovered cases for each country. Both data sources are updated daily retrieved directlt from the Github link and both datasets includes COVID data for Australia, Brazil, India, Russia, South Africa and United States, which is suitable for this notebook's purpose. Note that since these raw data is sourced directly from the Github link that is managed by the two organisations, hence both raw data have been **preserved immutability**.

Since we would like to estimate the number of true infection cases in order to determine how deadly or what's the risk of Covid. The 5 other datasets are sourced from ['Our World in Data'](https://ourworldindata.org/covid-models) which is data of the epidemiological models for COVID-19 that allows us to choose the most appropriate model to estimate the infected cases in each country. It is also important to note that these model files have not been preserved thier immutability as we would like them to, since they require downloads and re-upload on our group Github.




## Import Libraries
- *These sections below are written on the 4th of Nov*

The code below imports essential packages in order to conduct analysis. Main libraries used in this notebook are Pandas, Matplotlib, Plotly, Numpy, Seaborn snd Datetime. 


In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime
from math import log10, floor
import sympy as sp 
from plotly.subplots import make_subplots
from google.colab import files

The libraries have been imported in the code above.

Now we need to install Orca package, it converts the interactive plotly graphs into static images, so that the plotly charts are able to display when downloaded either in pdf or ipynb files when uploaded into Github. This package installation is retreived from: https://plotly.com/python/orca-management/

In [2]:
'''!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
'''

'!pip install plotly>=4.7.1\n!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca\n!chmod +x /usr/local/bin/orca\n!apt-get install xvfb libgtk2.0-0 libgconf-2-4\n'

All required libraies and packages are installed in the section above. Now we can continue with reading in all the raw data that will be used in the following analysis.

## Load in Datasets


We need to load in our data to begin the analysis. The code below reads in the global COVID-19 data from the "Our World in Data" source straight from the GitHub repository to ensure the most updated version is being used in this analysis as updates are made daily. 

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

The file sourced from "Our World in Data" is read into this notebook as *'df'* as it is the main dataset that our analysis is based on. 

Same step is done to load in the "John Hopkins" dataset , as the "Our World in Data" does not have the recovered data for each country that we would like explore.

In [4]:
jh_recovered=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

Note that the data has been loaded above.  John Hopkins data for recovered cases named as *'jh_recovered'*.

Next we will load in the datasets for the estimations of the infection cases by 4 different models, this is for the purpose of exploring the different models.

In [5]:
estimate=pd.read_csv('https://raw.github.sydney.edu.au/swan9801/R13B-Group4-COVID/master/Checkpoint%202/Model%20Datasets/daily-new-estimated-infections-of-covid-19.csv?token=AAABCWMB432J6ZRQE7SCXDC7VUFZO')

The dataset for all 4 models's estimation is loaded in as above  and assigned to *'estimate'*.

The following line will load in 4 models (data consisting the model's upper and lower bound for estimations) that we used for calculating the Infection Fatality Rates.

In [6]:
icl=pd.read_csv('https://raw.github.sydney.edu.au/swan9801/R13B-Group4-COVID/master/Checkpoint%202/Model%20Datasets/daily-new-estimated-covid-19-infections-icl-model.csv?token=AAABCWN3WZDCRRBJZWTXG227VUF5Y')
ihme=pd.read_csv('https://raw.github.sydney.edu.au/swan9801/R13B-Group4-COVID/master/Checkpoint%202/Model%20Datasets/daily-new-estimated-covid-19-infections-ihme-model.csv?token=AAABCWKQ7LENULGBAYZAQMK7VUF4A')
yyg=pd.read_csv('https://raw.github.sydney.edu.au/swan9801/R13B-Group4-COVID/master/Checkpoint%202/Model%20Datasets/daily-new-estimated-covid-19-infections-yyg-model.csv?token=AAABCWMEORQEATLQ3EEU7YC7VUFXK')
ishtm=pd.read_csv('https://raw.github.sydney.edu.au/swan9801/R13B-Group4-COVID/master/Checkpoint%202/Model%20Datasets/daily-new-estimated-covid-19-infections-lshtm-mode.csv?token=AAABCWK62GZMUFVRWFNUPFS7VUFV2')

All 4 model datasets are loaded in separately and each named after their model name,including *ICL,IHME* and *YYG* Infection cases estimation model.

## Data Engineering

### Data Munging
First we will go through the process of data munging to transform into another data format to make it appropriate for data analysis. Ideally, we would like to work with only one dataset as this makes data wrangling and exploration easier than working on two separate datasets, hence *jh_recovered* is transformed so that it can merge with *df*.

We print out the first five rows for *jh_recovered* dataset.

In [7]:
jh_recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,32746,32789,32842,32842,32842,32852,32879,32977,33045,33058,33058,33064,33114,33118,33308,33354,33447,33516,33561,33614,33760,33790,33824,33831,34010,34023,34129,34150,34217,34237,34239,34258,34321,34326,34342,34355,34362,34440,34440,34446
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7732,7847,8077,8342,8536,8675,8825,8965,9115,9215,9304,9406,9500,9585,9675,9762,9864,9957,10001,10071,10167,10225,10341,10395,10466,10548,10654,10705,10808,10893,11007,11097,11189,11246,11367,11473,11578,11696,11861,12002
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,36063,36174,36282,36385,36482,36578,36672,36763,36857,36958,37067,37170,37170,37382,37492,37603,37603,37856,37971,38088,38215,38346,38482,38618,38788,38932,39095,39273,39444,39635,39635,40014,40201,40395,40577,40577,41001,41244,41510,41783
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1265,1432,1432,1540,1540,1540,1615,1615,1715,1715,1814,1814,1814,1928,1928,2011,2011,2057,2057,2057,2273,2273,2470,2470,2729,2729,2729,2957,3029,3144,3260,3377,3475,3475,3548,3627,3734,3858,3858,4043
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1833,1941,2082,2215,2436,2577,2591,2598,2598,2635,2685,2716,2743,2744,2761,2801,2928,3012,3022,3030,3031,3037,3040,3305,3384,3461,3508,3530,3647,3693,3736,4107,4523,4920,5172,5230,5266,5350,5626,5647


From the output above we can observe that the John Hopkin's dataset has split the dates into separte column, hence we need to transform the dataframe from wide to long format using the melt function so that the dataframe is consistent with the *Our World in data*, suitable for later analysis.

In [8]:
jh_recovered2=jh_recovered.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name = 'date', value_name = 'total_recovered')
jh_recovered2.head()

,Province/State,Country/Region,Lat,Long,date,total_recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In the above code we reformatted the John Hopkin's file by melting the data under muiltiple columns into a new column *recovered* that records the recovered cases for each day and is grouped by the *date*. The new dataframe is named as jh_recovered2 as shown in the output above.


Next we will need to change the date column for *jh_recovered2* and *df* into dateformat in preparation for the merging process, repeat same for both datasets.

In [9]:
jh_recovered2['date']=pd.to_datetime(jh_recovered2['date'], errors='ignore')
df['date']=pd.to_datetime(df['date'], errors='ignore')

The date format has been rectified with the above code. 


We noticed the country names are inconsistent between the 2 files for United States hence we will need to replace the names in the *John Hopkins* file to match the *Our World in Data*. The other country names are consistent.


In [10]:
jh_recovered2['Country/Region'] = jh_recovered2['Country/Region'].replace(['US'],'United States')

Before we can start merging, as the first dataset as the *jh_recovered* data has it's recovered cases split up based on the "Province/State" as opposed to the *df* data. Therefore, our first step is to ungroup the *jh_recovered* "Province/State" to result in a sum of recovered cases for each country.

In [11]:
jh_recovered2_grouped=jh_recovered2.groupby(["Country/Region","date"]).sum()

Now the data has one daily value grouped by each country which is suitable for the merging process.



In order to join the datasets together, we need to find common variables to join them, ensuring the data matches up accordingly. The code below changes the format of the Johns Hopkins recovery dataset from wide to long by changing the dates that are currently as column names to their own column.
Merge the two data sets based on the 'location' & 'date' variables in the 'Our World data' and 'Country/Region' & 'date' variables in the John Hopkins data.

In [12]:
merge1=df.merge(jh_recovered2_grouped, how='inner', left_on=["location", "date"], right_on=["Country/Region","date"])

In [13]:
merge1.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,Lat,Long,total_recovered
0,AFG,Asia,Afghanistan,2020-01-22,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0
1,AFG,Asia,Afghanistan,2020-01-23,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0
2,AFG,Asia,Afghanistan,2020-01-24,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0
3,AFG,Asia,Afghanistan,2020-01-25,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0
4,AFG,Asia,Afghanistan,2020-01-26,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0


From the above output, we can see the merge was successful, a new column for recovered cases is added at the end of the dataframe.

The *recovered* column is a cumulative value of the cases recovered. Now we need to add a new column called 'new_recover' that calculates the daily cases using the 'recovered' variable, so that each value in 'new_recover' corresponds to a daily value.

In [14]:
new_recover=[0]
rec=list(merge1["total_recovered"])
loc=list(merge1["location"])
for j in range(len(merge1)-1):
  if loc[j+1] == loc[j]:
    new_recover.append(rec[j+1]-rec[j])
  else:
    new_recover.append(0)

merge1["new_recover"]=new_recover
merge1.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,Lat,Long,total_recovered,new_recover
0,AFG,Asia,Afghanistan,2020-01-22,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0,0
1,AFG,Asia,Afghanistan,2020-01-23,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0,0
2,AFG,Asia,Afghanistan,2020-01-24,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0,0
3,AFG,Asia,Afghanistan,2020-01-25,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0,0
4,AFG,Asia,Afghanistan,2020-01-26,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,33.93911,67.709953,0,0


New column *new_recover* has been appended shown in the output above. A list is created and for each country we calculates the daily recovered cases starting with 0 for each country. Data munging has been completed.


In [15]:
merge1['new_recover'].min()

-16298

As per the output above we can see that there exist some negative values this is due to negative corrections and adjustments made by the data source.

Now we will subset the data so that it only includes the 6 countries that our data analysis is focused on - Australia, Brazil, India, Russia, South Africa and United States.
To do this first we will create a list containing the 6 countries this help improve the readability and assist in further analysis.

In [16]:
countries=sorted(['United States','India','Russia','Brazil','South Africa', 'Australia'])

In [17]:
covid=merge1[merge1['location'].isin(countries)]
covid.shape

(1730, 53)

The data is subsetted as the output above and renamed to *covid*. We can see that the subset consists of 1712 rows and 53 columns now.

The other 5 datasets will also needed to be subsetted so that we only include the 6 countries we are conducting analysis on. This will present a simpler view when the group conducts analysis.




In [18]:
estimate=estimate[estimate['Entity'].isin(countries)]
icl=icl[icl['Entity'].isin(countries)]
ihme=ihme[ihme['Entity'].isin(countries)]
lshtm=ishtm[ishtm['Entity'].isin(countries)]
yyg=yyg[yyg['Entity'].isin(countries)]

The above code has only kept the rows that belongs to the countries we are interested in.

Now we can continue to the data cleaning stage.

### Data Cleaning

First, let's check if the the data types for each variables are appropriate.

In [19]:
covid.dtypes

iso_code                                      object
continent                                     object
location                                      object
date                                  datetime64[ns]
total_cases                                  float64
new_cases                                    float64
new_cases_smoothed                           float64
total_deaths                                 float64
new_deaths                                   float64
new_deaths_smoothed                          float64
total_cases_per_million                      float64
new_cases_per_million                        float64
new_cases_smoothed_per_million               float64
total_deaths_per_million                     float64
new_deaths_per_million                       float64
new_deaths_smoothed_per_million              float64
icu_patients                                 float64
icu_patients_per_million                     float64
hosp_patients                                f

No change in data types are required, since the data types are correct with numerical values in float64 type, date in datetime format and country codes and country names being objects.

Now we need to consider dealing with missing values by looking at the counts of the number of NA values in the dataset.

In [20]:
covid.isna().sum(axis = 0)

iso_code                                 0
continent                                0
location                                 0
date                                     0
total_cases                             87
new_cases                               31
new_cases_smoothed                      34
total_deaths                           299
new_deaths                              31
new_deaths_smoothed                     34
total_cases_per_million                 87
new_cases_per_million                   31
new_cases_smoothed_per_million          34
total_deaths_per_million               299
new_deaths_per_million                  31
new_deaths_smoothed_per_million         34
icu_patients                          1730
icu_patients_per_million              1730
hosp_patients                         1730
hosp_patients_per_million             1730
weekly_icu_admissions                 1730
weekly_icu_admissions_per_million     1730
weekly_hosp_admissions                1730
weekly_hosp

From the output above, we can see that for many variables we have NA / missing values. These variables include total_cases, new_cases, new_cases_smoothed...number of icu_patients,female_smokers,handwashing_facilities naming a few. This is due to technology inabilities or countries that does not report certain information or other factors that impede the data collection for these information. 

Since in this stage of the analysis we are looking at the confirmed,death, and recovered cases therefore, in this stage we will need to replace those NAs with 0. However, we will leave the other varoables untouched such as handwashing_facilities, icu_patients, stringent_index naming a few, since it would not be sensible to replace these variables with 0 as they are data that's not reported, we cannot interpret them as 0.

In [21]:
cols = ['total_cases','new_cases','new_cases_smoothed','total_deaths','new_deaths','new_deaths_smoothed','total_cases_per_million','new_cases_per_million','new_cases_smoothed_per_million','total_deaths_per_million','new_deaths_per_million','new_deaths_smoothed_per_million','total_tests','new_tests','total_tests_per_thousand','new_tests_per_thousand','new_tests_smoothed','new_tests_smoothed_per_thousand','tests_per_case','total_recovered','new_recover']
covid[cols]=covid[cols].fillna(0.0001)
covid[cols]=covid[cols].replace(0,0.0001)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



With the above code,we have replaced the selected columns with 0. Check if this has been effective by counting the total sum of NAs.

In [22]:
covid.isna().sum(axis = 0)

iso_code                                 0
continent                                0
location                                 0
date                                     0
total_cases                              0
new_cases                                0
new_cases_smoothed                       0
total_deaths                             0
new_deaths                               0
new_deaths_smoothed                      0
total_cases_per_million                  0
new_cases_per_million                    0
new_cases_smoothed_per_million           0
total_deaths_per_million                 0
new_deaths_per_million                   0
new_deaths_smoothed_per_million          0
icu_patients                          1730
icu_patients_per_million              1730
hosp_patients                         1730
hosp_patients_per_million             1730
weekly_icu_admissions                 1730
weekly_icu_admissions_per_million     1730
weekly_hosp_admissions                1730
weekly_hosp

The missing values have been successfully cleaned and replaced as shown in the counts above.

Now let's continue to check if there exists any null value in the models dataset (including *estimate*, *yyg*,*ihme*,*icl*,*ishtm*)



In [23]:
estimate.isna().sum(axis = 0)


Entity                                                                                             0
Code                                                                                               0
Date                                                                                               0
Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned)                  34
Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned) Annotations    1773
Daily new estimated infections of COVID-19 (ICL, mean)                                           197
Daily new estimated infections of COVID-19 (IHME, mean)                                          221
Daily new estimated infections of COVID-19 (YYG, mean)                                           390
Daily new estimated infections of COVID-19 (LSHTM, median)                                       851
dtype: int64

As printed above, we can see each infection case estimation model has a certain degree of incomplete data or null values. This is because the models each have a different update frequency approximately every 2 to 3 weeks, hence the data available to us is lagged in the dates. With some models ceased updates since August, this may be due to various reasons. Therefore, in this situation we should not replace the missing values with 0s which could potentially decrease the accuracy or magnifying the uncertainties during the data cleaning stage. We will leave these values untouched so that they are not included in our later calculations of COVID-19's death rate.

- *(This section below is added on 4th of Nov)*


Same process is required for the other 4 model estimation datasets.
We will first check the data types of the variables in the datasets.

In [24]:
print(ishtm.dtypes)
print(estimate.dtypes)

Entity                                                                                           object
Code                                                                                             object
Date                                                                                             object
Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned)                float64
Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned) Annotations     object
Daily new estimated infections of COVID-19 (LSHTM, median)                                      float64
Daily new estimated infections of COVID-19 (LSHTM, lower)                                       float64
Daily new estimated infections of COVID-19 (LSHTM, upper)                                       float64
dtype: object
Entity                                                                                           object
Code                                              

The date variable needs to be changed into dateformat.

In [25]:
icl['Date']=pd.to_datetime(icl['Date'], errors='ignore')
ihme['Date']=pd.to_datetime(ihme['Date'], errors='ignore')
yyg['Date']=pd.to_datetime(yyg['Date'], errors='ignore')
ishtm['Date']=pd.to_datetime(ishtm['Date'], errors='ignore')
estimate['Date']=pd.to_datetime(estimate['Date'], errors='ignore')

All four estimation model datasets have been changed into dateformat.



Now for the ease of analysis and readability of the codes, we will replace the variable names that are untidy with short and meaningful variable names.

In [26]:
estimate.rename(columns={'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned)':'daily_new_confirmed',
                    'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned) Annotations':'annotations',
                    'Daily new estimated infections of COVID-19 (ICL, mean)':'icl_mean',
                    'Daily new estimated infections of COVID-19 (IHME, mean)':'ihme_mean',
                    'Daily new estimated infections of COVID-19 (YYG, mean)':'yyg_mean',
                    'Daily new estimated infections of COVID-19 (LSHTM, median)':'lshtm_median'},inplace=True)
ihme.rename(columns={'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned)':'daily_new_confirmed',
                    'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned) Annotations':'annotations',
                    'Daily new estimated infections of COVID-19 (IHME, mean)':'ihme_mean',
                    'Daily new estimated infections of COVID-19 (IHME, lower)':'ihme_lower',
                    'Daily new estimated infections of COVID-19 (IHME, upper)':'ihme_upper'},inplace=True)

icl.rename(columns={'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned)':'daily_new_confirmed',
                    'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned) Annotations':'annotations',
                    'Daily new estimated infections of COVID-19 (ICL, mean)':'icl_mean',
                    'Daily new estimated infections of COVID-19 (ICL, lower)':'icl_lower',
                    'Daily new estimated infections of COVID-19 (ICL, upper)':'icl_upper'},inplace=True)

yyg.rename(columns={'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned)':'daily_new_confirmed',
                    'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned) Annotations':'annotations',
                    'Daily new estimated infections of COVID-19 (YYG, mean)':'yyg_mean',
                    'Daily new estimated infections of COVID-19 (YYG, lower)':'yyg_lower',
                    'Daily new estimated infections of COVID-19 (YYG, upper)':'yyg_upper'},inplace=True)

ishtm.rename(columns={'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned)':'daily_new_confirmed',
                    'Daily new confirmed cases due to COVID-19 (rolling 7-day average, right-aligned) Annotations':'annotations',
                    'Daily new estimated infections of COVID-19 (LSHTM, median)':'lshtm_median',
                    'Daily new estimated infections of COVID-19 (LSHTM, lower)':'lshtm_lower',
                    'Daily new estimated infections of COVID-19 (LSHTM, upper)':'lshtm_upper'},inplace=True)

In the above code the long variable names for all 4 datasets have been replaced with a shorter name for later analysis.

## Outliers 

- *This section is written on the 6th of Nov*

Now moving on to checking existing outliers in the dataset. To visualise the outliers we will use box plots for the variables that will be used in later analysis and are not a constant value, this includes *total_cases, new_cases, new_cases_smoothed, total_deaths, new_deaths, new_deaths_smoothed,total_recovered and new_recover.*

In [62]:
variable_list=['total_cases','new_cases','new_cases_smoothed','total_deaths','new_deaths','new_deaths_smoothed','total_recovered','new_recover','total_tests','new_tests','new_tests_smoothed']
box1 = make_subplots(rows=4, cols=2)
ntuple=([1,1],[1,2],[1,2],[2,1],[2,2],[2,2],[4,1],[4,2],[3,1],[3,2],[3,2])
for i,j in enumerate(range(len(ntuple))):
  box1.add_trace(
    go.Box(y=covid[variable_list[i]],name=variable_list[i]),
      row=ntuple[j][0], col=ntuple[j][1])
  
box1.show()

From the box plot outputted above, we can observe that all the data set are rightly skewed and for the variables *new_deaths,new recover and new cases* there exists some outliers on the larger end. However, these outliers does not exist as we look at the corresponding smoothed version. As these outliers reconciles data from previous days and is not incorrect, we can keep this entry in the dataset. Number of cases on a given day does not necessarily represent the actual number on that date since it has a long reporting chain, both large and negative values can be made to a country's entire time series to correct values retrospectively.This is just an example of how time variation and delays occur when studying an ongoing pandemic with information provided by https://github.com/owid/covid-19-data/tree/master/public/data (Our World in Data Source)

## Prepare data for succeeding process notebooks

To eliminate duplicated codes for data cleaning and transforming in the succeeding process notebooks. We will create csv files for the cleaned data sets ready for the group to import and conduct further analysis.

When creating the csv files we will intentionally remove. the index column since we noticed that this would cause a duplicate 'unnamed' column with reading the files in.

In [29]:
covid.to_csv('covid.csv', index=False) 
files.download('covid.csv')

estimate.to_csv('estimate.csv', index=False) 
files.download('estimate.csv')
yyg.to_csv('yyg.csv', index=False) 
files.download('yyg.csv')
ishtm.to_csv('ishtm.csv', index=False) 
files.download('ishtm.csv')
icl.to_csv('icl.csv', index=False) 
files.download('icl.csv')
ihme.to_csv('ihme.csv', index=False) 
files.download('ihme.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

"\nestimate.to_csv('estimate.csv', index=False) \nfiles.download('estimate.csv')\nyyg.to_csv('yyg.csv', index=False) \nfiles.download('yyg.csv')\nishtm.to_csv('ishtm.csv', index=False) \nfiles.download('ishtm.csv')\nicl.to_csv('icl.csv', index=False) \nfiles.download('icl.csv')\nihme.to_csv('ihme.csv', index=False) \nfiles.download('ihme.csv')"

As shown above, 5 cleaned files have been created and downloaded. These will be uploaded to Github. Note that this method violates the aim of preserving immutability of the data, hence we will only be temporary doing this, for the final product notebook this step will be eliminated to reduce the risk of changing the data sets and eliminate the uncertainties.